En este notebook se realiza una aproximación al problema utilizando un preprocesamiento simplificado donde se eliminan algunas variables poco relevantes, se aplica codificación One-Hot sobre variables categóricas y se entrena un modelo de Regresión Logística sin normalizar los datos numéricos, el enfoque de esta aproximación busca una solución rápida y reproducible con un modelo lineal ligero

In [ ]:
#Carga datos desde Google Drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


train_path = '/content/drive/MyDrive/Proyecto IA/Pruebas_Saber_Pro/train.csv'
test_path = '/content/drive/MyDrive/Proyecto IA/Pruebas_Saber_Pro/test.csv'


df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

# Toma una muestra para evitar uso excesivo de memoria
df_train = df_train.sample(50000, random_state=42)

# Elimina filas sin etiqueta
df_train = df_train.dropna(subset=['RENDIMIENTO_GLOBAL'])

# Imputa valores nulos
for df in [df_train, df_test]:
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna('Desconocido')
        else:
            df[col] = df[col].fillna(df[col].median())

# Guarda ID
test_ids = df_test['ID']

# Etiqueta test como 'desconocido'
df_train['source'] = 'train'
df_test['RENDIMIENTO_GLOBAL'] = 'desconocido'
df_test['source'] = 'test'
df_all = pd.concat([df_train, df_test])

# Codificación One-Hot para todas las columnas categóricas
cat_cols = df_all.select_dtypes(include='object').columns.tolist()
cat_cols.remove('RENDIMIENTO_GLOBAL')
cat_cols.remove('source')

df_all = pd.get_dummies(df_all, columns=cat_cols, drop_first=True)



df_train = df_all[df_all['source'] == 'train'].drop(columns=['source'])
df_test = df_all[df_all['source'] == 'test'].drop(columns=['source', 'RENDIMIENTO_GLOBAL'])

X = df_train.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y = df_train['RENDIMIENTO_GLOBAL']
X_test = df_test.drop(columns=['ID'])

# Entrenar el modelo
model = LogisticRegression(max_iter=1000)
model.fit(X, y)

# Predicción
preds = model.predict(X_test)

# Genera archivo para conocer el resultado del modelo
submission = pd.DataFrame({'ID': test_ids, 'RENDIMIENTO_GLOBAL': preds})
submission.to_csv('submission.csv', index=False)
print(" submission.csv generado correctamente.")





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 submission.csv generado correctamente.
